In [1]:
import tensorflow as tf
import json
import os
import sys
import keras


In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(1,(3, 3),strides=(1,1), activation='relu', padding='valid',input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.save("model1.keras")
model.summary()

e:\Apps_and_Programs\anaconda\envs\CNN_TF_GPU\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 1)      │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 169)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        10,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,300 (51.95 KB)

 Trainable params: 13,300 (51.95 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import tensorflow as tf
import json

MODEL_PATH = "model1.keras"
OUT_PATH = "hardware_model.json"

SUPPORTED_LAYERS = {
    "Conv2D": {
        "supported": ["filters", "kernel_size", "strides", "activation"],
        "valid_activations": ["relu", None],
    },
    "MaxPooling2D": {"supported": ["pool_size", "strides"]},
    "Dense": {
        "supported": ["units", "activation"],
        "valid_activations": ["relu", "sigmoid", "softmax"],
    },
    "Flatten": {"supported": []},
}

def get_tensor_shape(tensor):
    if tensor is None:
        return None
    return [int(s) if s is not None else None for s in tensor.shape]

def infer_layer_shapes(model):
    """Run a dummy forward pass through each layer to infer in/out shapes."""
    # Dynamically detect input shape
    if hasattr(model, "inputs") and model.inputs:
        shape = model.inputs[0].shape
    elif hasattr(model, "input_shape") and model.input_shape:
        shape = model.input_shape
    else:
        raise ValueError("❌ Could not detect input shape for model.")

    # Replace None (batch dim) with 1
    shape = [1 if s is None else s for s in shape]
    dummy_input = tf.zeros(shape)

    print(f"⚙️ Running dummy inference with shape: {shape}")
    layer_shapes = []
    x = dummy_input
    for layer in model.layers:
        in_shape = get_tensor_shape(x)
        x = layer(x)
        out_shape = get_tensor_shape(x)
        layer_shapes.append((in_shape, out_shape))
    return layer_shapes

def scalarize(param):
    """Convert tuple/list params (like (3,3)) into single scalar (e.g., 3)."""
    if isinstance(param, (list, tuple)):
        if len(set(param)) == 1:
            return param[0]
        else:
            raise ValueError(f"Hardware only supports square params, got {param}")
    return param

def extract_layer_info(layer, in_shape, out_shape):
    """Extract simplified hardware-friendly parameters."""
    ltype = layer.__class__.__name__
    cfg = layer.get_config()
    layer_data = {"name": layer.name, "type": ltype}

    # --- Input shape parsing ---
    if in_shape and len(in_shape) == 4:
        _, in_h, in_w, in_c = in_shape
        layer_data["input_shape"] = [in_h, in_w]
        layer_data["input_channels"] = in_c
    elif in_shape and len(in_shape) == 2:
        _, n = in_shape
        layer_data["input_shape"] = [n]
        layer_data["input_channels"] = 1

    # --- Output shape parsing ---
    if out_shape and len(out_shape) == 4:
        _, out_h, out_w, out_c = out_shape
        layer_data["output_shape"] = [out_h, out_w]
        layer_data["output_channels"] = out_c
    elif out_shape and len(out_shape) == 2:
        _, n = out_shape
        layer_data["output_shape"] = [n]
        layer_data["output_channels"] = n

    # --- Supported params ---
    for key in SUPPORTED_LAYERS.get(ltype, {}).get("supported", []):
        if key in cfg:
            val = cfg[key]
            keyname = "stride" if key == "strides" else key
            layer_data[keyname] = scalarize(val)

    # --- Activation ---
    act = cfg.get("activation", None)
    if act and act not in SUPPORTED_LAYERS.get(ltype, {}).get("valid_activations", [act]):
        raise ValueError(f"Unsupported activation {act} in layer {layer.name}")
    if act:
        layer_data["activation"] = act

    return layer_data

def main():
    print("🔍 Loading model...")
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)

    print("⚙️ Running dummy inference to collect shapes...")
    layer_shapes = infer_layer_shapes(model)

    layers_out = []
    for layer, (in_shape, out_shape) in zip(model.layers, layer_shapes):
        layers_out.append(extract_layer_info(layer, in_shape, out_shape))

    model_info = {"model_name": model.name, "layers": layers_out}
    with open(OUT_PATH, "w") as f:
        json.dump(model_info, f, indent=2)

    print(f"\n✅ Hardware model saved to {OUT_PATH}")

if __name__ == "__main__":
    main()


🔍 Loading model...
⚙️ Running dummy inference to collect shapes...
⚙️ Running dummy inference with shape: [1, 28, 28, 1]

✅ Hardware model saved to hardware_model.json


In [ ]:
import json

# --------------------------- Configuration --------------------------- #

INPUT_FILE = "software_model.json"   # output of Stage 1
OUTPUT_FILE = "hardware_model.json"  # final output JSON for Stage 2

GLOBAL_DEFAULTS = {
    "DATA_WIDTH": 16,
    "FRACTION_SIZE": 14,
    "SIGNED": 1,
    "GUARD_TYPE": 2
}

# --------------------------- Block Builders --------------------------- #

def make_imgbuf_block(name, layer, idx, prev_out):
    """Creates ImageBufferChnl block JSON."""
    k = layer.get("kernel_size", layer.get("pool_size"))
    return {
        name: {
            "module": "ImageBufferChnl",
            "parameters": {
                "KERNEL_SIZE": k,
                "DATA_WIDTH": "DATA_WIDTH",
                "COLUMN_NUM": layer["input_shape"][0],
                "ROW_NUM": layer["input_shape"][1],
                "STRIDE": layer["stride"],
                "CHANNELS": layer["input_channels"]
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": "data_valid" if idx == 1 else f"{prev_out}_valid", "width": 1},
                "data_in": {"name": "data_in" if idx == 1 else prev_out, "width": "DATA_WIDTH"}
            },
            "outputs": {
                "data_out": {"name": f"{name}_out", "width": "DATA_WIDTH"},
                "kernel_out": {"name": f"kernel_out_{name}", "width": f"DATA_WIDTH * {k} * {k}"},
                "kernel_valid": {"name": f"kernel_valid_{name}", "width": 1}
            }
        }
    }

def make_conv_block(name, layer, ibuf_name, idx):
    """Creates ConvChnl block JSON (now points to .mif files)."""
    k = layer["kernel_size"]
    return {
        name: {
            "module": "ConvChnl",
            "parameters": {
                "KERNEL_SIZE": k,
                "CHANNELS": layer["input_channels"],
                "DATA_WIDTH": "DATA_WIDTH",
                "FRACTION_SIZE": "FRACTION_SIZE",
                "SIGNED": "SIGNED",
                "ACTIVATION": layer.get("activation", "none"),
                "GUARD_TYPE": "GUARD_TYPE"
            },
            "files": {
                "weights_file": f"conv_L{idx}_weights.mif",
                "biases_file": f"conv_L{idx}_biases.mif"
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": f"kernel_valid_{ibuf_name}", "width": 1},
                "kernel_in": {"name": f"kernel_out_{ibuf_name}", "width": f"DATA_WIDTH * {k} * {k}"}
            },
            "outputs": {
                "conv_out": {"name": f"{name}_out", "width": "DATA_WIDTH"},
                "conv_valid": {"name": f"{name}_valid", "width": 1}
            }
        }
    }

def make_maxpool_block(name, layer, ibuf_name):
    """Creates MaxpoolChnl block JSON."""
    k = layer["pool_size"]
    return {
        name: {
            "module": "MaxpoolChnl",
            "parameters": {
                "KERNEL_SIZE": k,
                "DATA_WIDTH": "DATA_WIDTH",
                "SIGNED": "SIGNED",
                "CHANNELS": layer["input_channels"]
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": f"kernel_valid_{ibuf_name}", "width": 1},
                "kernel_in": {"name": f"kernel_out_{ibuf_name}", "width": f"DATA_WIDTH * {k} * {k}"}
            },
            "outputs": {
                "maxp_out": {"name": f"{name}_out", "width": "DATA_WIDTH"},
                "maxp_valid": {"name": f"{name}_valid", "width": 1}
            }
        }
    }

def build_nn_block(prev_out_signal, dense_layers):
    """Creates the NN structure (Dense layers referencing .mif files)."""
    nn_layers = []
    for i, layer in enumerate(dense_layers, start=1):
        entry = {
            "name": f"dense_L{i}",
            "num_inputs": layer.get("input_shape", ["TO_BE_FILLED"])[0],
            "num_neurons": layer["units"],
            "activation": layer.get("activation", "TO_BE_FILLED"),
            "weights_file": f"weights_L{i}.mif",
            "biases_file": f"biases_L{i}.mif"
        }
        nn_layers.append(entry)

    return {
        "Build": True,
        "globals": {
            "DATA_WIDTH": "DATA_WIDTH",
            "FRACTION_SIZE": "FRACTION_SIZE",
            "SIGNED": "SIGNED"
        },
        "io": {
            "input_signal": prev_out_signal,
            "input_valid": f"{prev_out_signal}_valid",
            "output_signal": "nn_out",
            "output_valid": "nn_valid"
        },
        "layers": nn_layers
    }

def make_maxfinder_block(prev_out_signal, num_outputs):
    """Creates a fixed MaxFinder block as the final stage."""
    return {
        "maxFinder": {
            "module": "maxFinder",
            "parameters": {
                "NUM_INPUTS": num_outputs,
                "DATA_WIDTH": "DATA_WIDTH",
                "SIGNED": "SIGNED"
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": f"{prev_out_signal}_valid", "width": 1},
                "data_in": {"name": prev_out_signal, "width": f"DATA_WIDTH * {num_outputs}"}
            },
            "outputs": {
                "class_idx": {"name": "class_idx", "width": "log2(NUM_INPUTS)"},
                "valid": {"name": "class_valid", "width": 1}
            }
        }
    }

# --------------------------- Main Conversion --------------------------- #

def main():
    with open(INPUT_FILE, "r") as f:
        model = json.load(f)

    layers = model["layers"]
    globals_ = GLOBAL_DEFAULTS.copy()

    top = {
        "module": "CNN",
        "hardware parameters": globals_,
        "inputs": {
            "clock": 1,
            "sreset_n": 1,
            "data_in": "DATA_WIDTH",
            "data_valid": 1
        },
        "outputs": {
            "class_idx": "log2(NUM_INPUTS)",
            "class_valid": 1
        }
    }

    prev_out = "data_in"
    stage_idx = 1
    dense_layers = []

    for layer in layers:
        ltype = layer["type"]

        if ltype == "Conv2D":
            ibuf_name = f"ImageBuffer_Conv{stage_idx}"
            conv_name = f"Conv{stage_idx}"
            top.update(make_imgbuf_block(ibuf_name, layer, stage_idx, prev_out))
            top.update(make_conv_block(conv_name, layer, ibuf_name, stage_idx))
            prev_out = f"{conv_name}_out"
            stage_idx += 1

        elif ltype == "MaxPooling2D":
            ibuf_name = f"ImageBuffer_Maxpool{stage_idx}"
            pool_name = f"Maxpool{stage_idx}"
            top.update(make_imgbuf_block(ibuf_name, layer, stage_idx, prev_out))
            top.update(make_maxpool_block(pool_name, layer, ibuf_name))
            prev_out = f"{pool_name}_out"
            stage_idx += 1

        elif ltype == "Flatten":
            continue  # Flatten handled implicitly

        elif ltype == "Dense":
            dense_layers.append(layer)

    # Add NN + MaxFinder
    if dense_layers:
        top["NN"] = build_nn_block(prev_out, dense_layers)
        last_layer = dense_layers[-1]
        num_outputs = last_layer["units"]
        top.update(make_maxfinder_block("nn_out", num_outputs))
    else:
        # No NN present, direct connection to output
        top["outputs"] = {
            "data_out": prev_out,
            "data_valid": f"{prev_out}_valid"
        }

    final_json = {"Hardware_model": {"top": top}}

    with open(OUTPUT_FILE, "w") as f:
        json.dump(final_json, f, indent=2)

    print(f"✅ Hardware model written to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


✅ Hardware model written to hardware_model.json
